In [ ]:
#create dataframe

In [1]:
df_hotel = spark.createDataFrame(
    [
        (5001, 21),
        (5001, 22),
        (5002,21)
    ],
    ["hotel_id", "advertiser"]
)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1643453159812_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df_hotel.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+
|hotel_id|advertiser|
+--------+----------+
|    5001|        21|
|    5001|        22|
|    5002|        21|
+--------+----------+

In [3]:
df_hotel.createOrReplaceTempView("hotel")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sqlDF = spark.sql("SELECT * FROM hotel")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+----------+
|hotel_id|advertiser|
+--------+----------+
|    5001|        21|
|    5001|        22|
|    5002|        21|
+--------+----------+

In [5]:
df_clicks =spark.createDataFrame(
    [
        ("US",21,5001,100),
        ("US",22,5001,50),
        ("US",21,5002,10),
        ("IT",22,5001,5)
    ],
    ["country" ,"advertiser","hotel_id","clicks" ]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_clicks.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+--------+------+
|country|advertiser|hotel_id|clicks|
+-------+----------+--------+------+
|     US|        21|    5001|   100|
|     US|        22|    5001|    50|
|     US|        21|    5002|    10|
|     IT|        22|    5001|     5|
+-------+----------+--------+------+

In [7]:
df_clicks.createOrReplaceTempView("clicks")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
#To have a better visualization of the case, I added more data in this data frame starting from the 3rd row.
df_bookings =spark.createDataFrame(
    [
        ("US",21,5001,10,1000),
        ("DE",21,5002,1,200),
        ("US",23,5003,2,500),
        ("US",23,5003,2,1500),
        ("US",26,5005,6,600),
        ("DE",25,5003,2,500),
        ("IT",3,5009,50,700),
        ("IT",25,5003,30,1000),
        ("IT",26,5012,30,800),
        ("IT",26,5007,30,700),
    ],
    ["country" ,"advertiser","hotel_id","bookings","booking_volume"]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
df_bookings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+--------+--------+--------------+
|country|advertiser|hotel_id|bookings|booking_volume|
+-------+----------+--------+--------+--------------+
|     US|        21|    5001|      10|          1000|
|     DE|        21|    5002|       1|           200|
|     US|        23|    5003|       2|           500|
|     US|        23|    5003|       2|          1500|
|     US|        26|    5005|       6|           600|
|     DE|        25|    5003|       2|           500|
|     IT|         3|    5009|      50|           700|
|     IT|        25|    5003|      30|          1000|
|     IT|        26|    5012|      30|           800|
|     IT|        26|    5007|      30|           700|
+-------+----------+--------+--------+--------------+

In [36]:
df_bookings.createOrReplaceTempView("bookings")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

###  1.Total number of clicks per hotel

In [37]:
sqlDF = spark.sql("\
        SELECT hotel_id,sum(clicks) AS total_clicks FROM clicks\
        GROUP BY hotel_id\
        ORDER BY hotel_id\
")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|hotel_id|total_clicks|
+--------+------------+
|    5001|         155|
|    5002|          10|
+--------+------------+

###  2.Top advertiser per country in terms of booking volume

In [38]:
sqlDF = spark.sql("\
            SELECT advertiser,country,SUM(booking_volume) AS total_volume\
            FROM bookings\
            GROUP BY country,advertiser\
            ORDER BY country\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("total_volume_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------------+
|advertiser|country|total_volume|
+----------+-------+------------+
|        21|     DE|         200|
|        25|     DE|         500|
|         3|     IT|         700|
|        26|     IT|        1500|
|        25|     IT|        1000|
|        23|     US|        2000|
|        21|     US|        1000|
|        26|     US|         600|
+----------+-------+------------+

In [39]:
sqlDF = spark.sql("\
SELECT * \
      FROM (SELECT advertiser,country,total_volume,rank() over (partition by country order by total_volume DESC) as rank\
      FROM total_volume_table)\
WHERE rank=1")
sqlDF.show()
sqlDF.createOrReplaceTempView("max_volume_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------------+----+
|advertiser|country|total_volume|rank|
+----------+-------+------------+----+
|        25|     DE|         500|   1|
|        26|     IT|        1500|   1|
|        23|     US|        2000|   1|
+----------+-------+------------+----+

### 3.2nd advertiser per country in terms of booking volume

In [40]:
sqlDF = spark.sql(" \
SELECT * \
    FROM( SELECT advertiser,country,total_volume, rank() over (partition by country order by total_volume DESC) AS rank\
    FROM total_volume_table)\
WHERE rank=2\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("without_top_volume_table")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------------+----+
|advertiser|country|total_volume|rank|
+----------+-------+------------+----+
|        21|     DE|         200|   2|
|        25|     IT|        1000|   2|
|        21|     US|        1000|   2|
+----------+-------+------------+----+

### 4.Booking conversion rate (bookings / clicks) for all hotels in trivago.advertiser_hotels

In [41]:
sqlDF = spark.sql("\
    SELECT hotel_id, sum(clicks) AS total_clicks FROM clicks\
    GROUP BY hotel_id\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("total_clicks_per_hotel")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------+
|hotel_id|total_clicks|
+--------+------------+
|    5002|          10|
|    5001|         155|
+--------+------------+

In [42]:
sqlDF = spark.sql("\
    SELECT hotel_id, sum(bookings) AS total_bookings FROM bookings\
    GROUP BY hotel_id\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("total_bookings_per_hotel")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+
|hotel_id|total_bookings|
+--------+--------------+
|    5009|            50|
|    5012|            30|
|    5002|             1|
|    5007|            30|
|    5003|            36|
|    5005|             6|
|    5001|            10|
+--------+--------------+

In [43]:
sqlDF = spark.sql("\
    SELECT total_bookings_per_hotel.hotel_id\
    FROM total_bookings_per_hotel\
    UNION\
    SELECT total_clicks_per_hotel.hotel_id\
    FROM total_clicks_per_hotel\
")
sqlDF.show()
sqlDF.createOrReplaceTempView("full_hotel_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|hotel_id|
+--------+
|    5009|
|    5012|
|    5002|
|    5007|
|    5003|
|    5005|
|    5001|
+--------+

In [44]:
sqlDF = spark.sql("\
    SELECT full_hotel_id.hotel_id,total_bookings,total_clicks,total_bookings/NULLIF(total_clicks,0) AS booking_conversion_rate\
    FROM full_hotel_id\
    full JOIN total_bookings_per_hotel ON total_bookings_per_hotel.hotel_id = full_hotel_id.hotel_id\
    full JOIN total_clicks_per_hotel ON total_clicks_per_hotel.hotel_id = full_hotel_id.hotel_id\
")
sqlDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------+------------+-----------------------+
|hotel_id|total_bookings|total_clicks|booking_conversion_rate|
+--------+--------------+------------+-----------------------+
|    5009|            50|        null|                   null|
|    5012|            30|        null|                   null|
|    5002|             1|          10|                    0.1|
|    5007|            30|        null|                   null|
|    5003|            36|        null|                   null|
|    5005|             6|        null|                   null|
|    5001|            10|         155|    0.06451612903225806|
+--------+--------------+------------+-----------------------+